In [3]:
import sys
sys.path.append(r'C:\Users\nanda\OneDrive\Desktop\market-regime-detection')

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [104]:
from src.data_loader import fetch_data
from src.features import compute_features
from src.hmm_regime import fit_hmm,analyze_states,label_states
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [42]:
DATA_DIR = Path(r'C:\Users\nanda\OneDrive\Desktop\market-regime-detection\data\Equity_1min')
csv_files = list(DATA_DIR.glob('*.csv'))

df = pd.DataFrame()
for f in csv_files:
    df = pd.concat([df,pd.read_csv(f)],ignore_index=False,axis=0)


In [44]:
df['date'] = df['date'].apply(lambda x:x.strip('="'))

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_timedelta(df['time'])
df['timestamp'] = df['date']+df['time']

In [50]:
df.set_index('timestamp',inplace=True)
df = df.sort_index()
df.drop(columns=['date','time'],inplace=True)

In [52]:
df_D = df.reset_index()
df_D['date'] = df_D['timestamp'].dt.date
df_D = df_D.groupby('date').agg(
    {
        'open':'first',
        'high' : 'max',
        'low':'min',
        'close':'last'

    }
)

In [ ]:
X_unscaled = compute_features(df_D['close'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_unscaled)
train_X,test_X = train_test_split(X_scaled,shuffle=False,train_size=0.8)

array([[ 2.63678842,  0.66681511,  0.78940116, ...,  1.82177979,
         0.69553721, -1.15411501],
       [-1.25681053,  0.64315136,  0.85294906, ..., -0.18671564,
         0.8518567 , -1.06847229],
       [ 0.70981041,  0.52722899,  0.80230156, ..., -0.62656799,
         0.89155917, -0.94148526],
       ...,
       [-0.16373059,  0.04089306,  1.11277576, ...,  1.26148119,
         0.04379597, -0.98289421],
       [-0.15706551,  0.02096004,  1.11256366, ...,  0.72063289,
         0.12210371, -0.97997335],
       [-0.34290801,  0.03397757,  1.11624864, ...,  0.53659851,
         0.154127  , -1.00538596]], shape=(746, 7))

In [113]:
hmm_model = fit_hmm(train_X)
states = hmm_model.predict(train_X)
analysis = analyze_states(hmm_model,train_X[:,0],states)

In [114]:
analysis

,state,mean_return,volatility,sharpe,pct_time,n_obs
0,0,-30.192751,23.049206,-1.309926,25.871314,193
1,1,23.705735,7.562756,3.134537,20.777480,155
2,2,6.021119,12.888019,0.467187,53.351206,398


(746,)